<a href="https://colab.research.google.com/github/TobiasMazan/Demand-forecast-demo-project/blob/main/Copy_of_ann_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ANN REGRESSION

In [ ]:
# 1. Wczytaj biblioteki 

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# Installing required libraries
!pip install tensorflow
!pip install keras
import tensorflow as tf
# importing the libraries
from keras.models import Sequential
from keras.layers import Dense
from google.colab import files

# 2. Wczytaj dane

# To remove the scientific notation from numpy arrays
np.set_printoptions(suppress=True)

from google.colab import drive
drive.mount('/content/drive')

# Reading the cleaned numeric car prices data
CarPricesDataNumeric=pd.read_pickle('/content/drive/My Drive/Colab Notebooks/CarPricesData.pkl')

CarPricesDataNumeric.head()

     |████████████████████████████████| 462 kB 6.6 MB/s 
Mounted at /content/drive


,Age,KM,Weight,HP,MetColor,CC,Doors,Price
0,23.0,46986,1165.0,90,1,2000.0,3,13500
1,23.0,72937,1165.0,90,1,2000.0,3,13750
2,24.0,41711,1165.0,90,1,2000.0,3,13950
3,26.0,48000,1165.0,90,0,2000.0,3,14950
4,30.0,38500,1170.0,90,0,2000.0,3,13750


In [ ]:
# 3. Zdefiniuj zmienne przewidywane 
TargetVariable=['Price']
y=CarPricesDataNumeric[TargetVariable].values

### Standardization of data ###
TargetVarScaler=StandardScaler()
# Storing the fit object for later reference
TargetVarScalerFit=TargetVarScaler.fit(y)
# Generating the standardized values of y
y=TargetVarScalerFit.transform(y)

# 4. Zdefiniuj zmienne przewidujące
Predictors=['Age', 'KM', 'Weight', 'HP', 'MetColor', 'CC', 'Doors']
X=CarPricesDataNumeric[Predictors].values

PredictorScaler=StandardScaler()
PredictorScalerFit=PredictorScaler.fit(X)
X=PredictorScalerFit.transform(X) 

In [ ]:
# 5. Utwórz model ML

# Split the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
 
# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# Create ANN model
model = Sequential()

# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=5, input_dim=7, kernel_initializer='normal', activation='relu'))

# Defining the Second layer of the model- after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=5, kernel_initializer='normal', activation='tanh'))

# The output neuron is a single fully connected node since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
model.fit(X_train, y_train ,batch_size = 20, epochs = 50, verbose=1)

# Generating Predictions on testing data
Predictions=model.predict(X_test)

# Scaling the predicted Price data back to original price scale
Predictions=TargetVarScalerFit.inverse_transform(Predictions)
# Scaling the y_test Price data back to original price scale
y_test_orig=TargetVarScalerFit.inverse_transform(y_test)
# Scaling the test data back to original scale
Test_Data=PredictorScalerFit.inverse_transform(X_test)



(1004, 7)
(1004, 1)
(431, 7)
(431, 1)
Epoch 1/50
51/51 [==============================] - 1s 2ms/step - loss: 0.9837
Epoch 2/50
51/51 [==============================] - 0s 1ms/step - loss: 0.9418
Epoch 3/50
51/51 [==============================] - 0s 1ms/step - loss: 0.7202
Epoch 4/50
51/51 [==============================] - 0s 1ms/step - loss: 0.4125
Epoch 5/50
51/51 [==============================] - 0s 2ms/step - loss: 0.2696
Epoch 6/50
51/51 [==============================] - 0s 1ms/step - loss: 0.2242
Epoch 7/50
51/51 [==============================] - 0s 1ms/step - loss: 0.2033
Epoch 8/50
51/51 [==============================] - 0s 1ms/step - loss: 0.1907
Epoch 9/50
51/51 [==============================] - 0s 2ms/step - loss: 0.1804
Epoch 10/50
51/51 [==============================] - 0s 2ms/step - loss: 0.1721
Epoch 11/50
51/51 [==============================] - 0s 2ms/step - loss: 0.1653
Epoch 12/50
51/51 [==============================] - 0s 1ms/step - loss: 0.1602
Epoch 13/50

In [ ]:
# 6. Interpretuj model ML

# Tabela testowa
TestingData=pd.DataFrame(data=Test_Data, columns=Predictors)
TestingData['Price']=y_test_orig
TestingData['PredictedPrice']=Predictions
TestingData.head()

# Computing the absolute percent error
APE=100*(abs(TestingData['Price']-TestingData['PredictedPrice'])/TestingData['Price'])
TestingData['APE']=APE
 
print('The Accuracy of ANN model is:', 100-np.mean(APE))
TestingData.head()

The Accuracy of ANN model is: 91.20372861674056


,Age,KM,Weight,HP,MetColor,CC,Doors,Price,PredictedPrice,APE
0,59.0,80430.0,1065.0,110.0,1.0,1600.0,3.0,9950.0,9997.986328,0.482275
1,62.0,64797.0,1075.0,110.0,1.0,1600.0,5.0,7995.0,10057.074219,25.792048
2,59.0,130000.0,1135.0,72.0,1.0,2000.0,4.0,7500.0,8325.143555,11.001914
3,69.0,42800.0,1050.0,110.0,1.0,1600.0,3.0,9950.0,9190.569336,7.632469
4,65.0,47014.0,1015.0,86.0,1.0,1300.0,3.0,8950.0,8833.944336,1.296711


In [ ]:
# 7. Optymalizuj model ML

# Defining a function to find the best parameters for ANN with manual grid search (nested for-loops)
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[5, 10, 15, 20]
    epoch_list  =   [5, 10, 50, 100]
    
    import pandas as pd
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            # create ANN model
            model = Sequential()
            # Defining the first layer of the model
            model.add(Dense(units=5, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
 
            # Defining the Second layer of the model
            model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
 
            # The output neuron is a single fully connected node 
            # Since we will be predicting a single number
            model.add(Dense(1, kernel_initializer='normal'))
 
            # Compiling the model
            model.compile(loss='mean_squared_error', optimizer='adam')
 
            # Fitting the ANN to the Training set
            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
 
            MAPE = np.mean(100 * (np.abs(y_test-model.predict(X_test))/y_test))
            
            # printing the results of the current iteration
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'Accuracy:', 100-MAPE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), 100-MAPE]],
                                                                    columns=['TrialNumber', 'Parameters', 'Accuracy'] ))
    return(SearchResultsData)
 
 
######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 5 - epochs: 5 Accuracy: 42.97191368823725
2 Parameters: batch_size: 5 - epochs: 10 Accuracy: 41.28460074810642
3 Parameters: batch_size: 5 - epochs: 50 Accuracy: 57.722956865690925
4 Parameters: batch_size: 5 - epochs: 100 Accuracy: 49.57014208803877
5 Parameters: batch_size: 10 - epochs: 5 Accuracy: 36.47792956406935
6 Parameters: batch_size: 10 - epochs: 10 Accuracy: 38.93768591915217
7 Parameters: batch_size: 10 - epochs: 50 Accuracy: 59.96448533877317
8 Parameters: batch_size: 10 - epochs: 100 Accuracy: 70.5809184021158
9 Parameters: batch_size: 15 - epochs: 5 Accuracy: 34.11619381489558
10 Parameters: batch_size: 15 - epochs: 10 Accuracy: 38.3036941726486
11 Parameters: batch_size: 15 - epochs: 50 Accuracy: 47.62896927905149
12 Parameters: batch_size: 15 - epochs: 100 Accuracy: 64.04380689278362
13 Parameters: batch_size: 20 - epochs: 5 Accuracy: 38.49949991067623
14 Parameters: batch_size: 20 - epochs: 10 Accuracy: 40.33309926512369
15 Parameters: batch_

In [ ]:
# Fitting the tuned ANN to the Training set
model.fit(X_train, y_train ,batch_size = 20, epochs = 10, verbose=0)

# Generating Predictions on testing data
Predictions=model.predict(X_test)

# Scaling the predicted Price data back to original price scale
Predictions=TargetVarScalerFit.inverse_transform(Predictions)

# Scaling the y_test Price data back to original price scale
y_test_orig=TargetVarScalerFit.inverse_transform(y_test)

# Scaling the test data back to original scale
Test_Data=PredictorScalerFit.inverse_transform(X_test)

TestingData=pd.DataFrame(data=Test_Data, columns=Predictors)
TestingData['Price']=y_test_orig
TestingData['PredictedPrice']=Predictions
TestingData.head()

# Computing the absolute percent error
APE=100*(abs(TestingData['Price']-TestingData['PredictedPrice'])/TestingData['Price'])
TestingData['APE']=APE
 
print('The Accuracy of ANN model is:', 100-np.mean(APE))
TestingData.head()

The Accuracy of ANN model is: 90.747078766181


,Age,KM,Weight,HP,MetColor,CC,Doors,Price,PredictedPrice,APE
0,59.0,80430.0,1065.0,110.0,1.0,1600.0,3.0,9950.0,10157.687500,2.087312
1,62.0,64797.0,1075.0,110.0,1.0,1600.0,5.0,7995.0,10104.569336,26.386108
2,59.0,130000.0,1135.0,72.0,1.0,2000.0,4.0,7500.0,8365.256836,11.536758
3,69.0,42800.0,1050.0,110.0,1.0,1600.0,3.0,9950.0,9187.219727,7.666133
4,65.0,47014.0,1015.0,86.0,1.0,1300.0,3.0,8950.0,8751.447266,2.218466


In [ ]:
# ADDITION: Defining a function to find the best parameters for ANN with GridSearchCV

# Function to generate Deep ANN model 
def make_regression_ann(Optimizer_trial):
    from keras.models import Sequential
    from keras.layers import Dense
    
    model = Sequential()
    model.add(Dense(units=5, input_dim=7, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer=Optimizer_trial)
    return model

###########################################
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

# Listing all the parameters to try
Parameter_Trials={'batch_size':[10,20,30],
                      'epochs':[10,20],
                    'Optimizer_trial':['adam', 'rmsprop']
                 }

# Creating the regression ANN model
RegModel=KerasRegressor(make_regression_ann, verbose=0)

###########################################
from sklearn.metrics import make_scorer

# Defining a custom function to calculate accuracy
def Accuracy_Score(orig,pred):
    MAPE = np.mean(100 * (np.abs(orig-pred)/orig))
    print('#'*70,'Accuracy:', 100-MAPE)
    return(100-MAPE)

custom_Scoring=make_scorer(Accuracy_Score, greater_is_better=True)

#########################################
# Creating the Grid search space
# See different scoring methods by using sklearn.metrics.SCORERS.keys()
grid_search=GridSearchCV(estimator=RegModel, 
                         param_grid=Parameter_Trials, 
                         scoring=custom_Scoring, 
                         cv=5)

#########################################
# Measuring how much time it took to find the best params
import time
StartTime=time.time()

# Running Grid Search for different paramenters
grid_search.fit(X,y, verbose=1)

EndTime=time.time()
print("########## Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes')

print('### Printing Best parameters ###')
grid_search.best_params_

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


115/115 [==============================] - 1s 1ms/step - loss: 0.3230
Epoch 2/10
115/115 [==============================] - 0s 1ms/step - loss: 0.1672
Epoch 3/10
115/115 [==============================] - 0s 1ms/step - loss: 0.1289
Epoch 4/10
115/115 [==============================] - 0s 1ms/step - loss: 0.1132
Epoch 5/10
115/115 [==============================] - 0s 1ms/step - loss: 0.1060
Epoch 6/10
115/115 [==============================] - 0s 1ms/step - loss: 0.1025
Epoch 7/10
115/115 [==============================] - 0s 1ms/step - loss: 0.1001
Epoch 8/10
115/115 [==============================] - 0s 1ms/step - loss: 0.0982
Epoch 9/10
115/115 [==============================] - 0s 1ms/step - loss: 0.0966
Epoch 10/10
115/115 [==============================] - 0s 2ms/step - loss: 0.0955
###################################################################### Accuracy: 21.345814421987455
Epoch 1/10
115/115 [==============================] - 1s 1ms/step - loss: 1.1580
Epoch 2/10
115/115 

{'Optimizer_trial': 'adam', 'batch_size': 20, 'epochs': 10}